In [1]:
import pandas as pd
import datetime, pytz
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt
import json

import talib
import pandas_ta
import time

from importlib import reload


import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_indicator import Indicators



C:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [30]:
def boltrend(Close, window, std, minspread, emawindow):
    bbands = vbt.talib('BBANDS').run(Close, window, std, std)
    rel_spread = (pd.Series(bbands.upperband.to_numpy().flatten() / bbands.lowerband.to_numpy().flatten()).shift(1).fillna(0).to_numpy() - 1) * 100
    ema = vbt.talib('EMA').run(Close, emawindow).real.to_numpy()

    entries = bbands.upperband_crossed_below(Close).to_numpy().flatten() & (rel_spread > minspread).flatten() & (Close > ema).flatten()
    exits = Close < bbands.middleband

    return entries, exits

BolTrend = vbt.IndicatorFactory(
    class_name= 'BolTrend',
    short_name = 'BT',
    input_names = ["Close"] ,
    param_names = ["window", "std", "minspread", "emawindow"],
    output_names= ["entries", "exits"]
    ).from_apply_func(boltrend)

In [18]:
data  = vbt.BinanceData.download('BTCUSDT', start = datetime.datetime(2023, 1,1)).get('Close')

0it [00:00, ?it/s]

In [27]:
BolTrend.run(data,  100, 2.25, 0, 500)

(200, 1)
(200, 1)
(200,)
(200, 1)


In [28]:
params = {
    'backtest': {
        'start': '2017-10-1',
        'end': '2018-1-1',
        'length': {
            'weeks': 4  #durée pendant laquelle on optimise les paramètres
        },
        'forward': {
            'weeks': 2  #durée pendant laquelle on joue les paramètres optimisés
        },
        
    },
    'strat':{
        'download_params': {
            'symbols': 'BTCUSDT',
            'interval': '1h',
            'missing_index': 'drop'            
        },
        'get_params': {
            'column': ['Close']
        },
        'run_params': { 
            'window': {'start': 50, 'stop': 150, 'step': 10},
            'std': {'start': 1.5, 'stop': 3, 'step': 0.25},
            'minspread': {'start': 0, 'stop': 5, 'step': 1},
            'emawindow': {'start': 100, 'stop': 600, 'step': 100},
            'param_product': 'True'
        }, 
    },
    'portfolio': {'freq': 'H' ,
                  'fees' : 0.1/100,
                  'size_type': 'Percent',
                  'size': .1} #proportion du portefeuille à investir dans chaque trade
}



In [31]:
indics = Indicators()
test = ForwardTest(BolTrend, params)
test.backtest(init_cash=1000)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-10-29 00:00:00+00:00 and 2017-11-12 00:00:00+00:00 finished at 2023-07-19 17:02:56.327200
Cash: 1000.0


0it [00:00, ?it/s]

ConnectionError: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out.

In [ ]:
pf = test.aggregate_results ()
pf.stats()